## Cost - Unit labor cost
## Efficiency - productivity

In [53]:
%pip install --upgrade altair vega_datasets

Note: you may need to restart the kernel to use updated packages.


In [54]:
import pandas as pd
import altair as alt
import numpy as np

df = pd.read_csv("../data/Processed/efficiency_productivity/efficiency_productivity.csv")

# compute yearly average labor productivity and efficiency
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df_year = df.groupby('Year', as_index=False)[['Labor productivity','Unit labor costs']].mean()
print(df_year)

    Year  Labor productivity  Unit labor costs
0   2013           102.84175          88.68700
1   2014           102.54325          91.29650
2   2015           100.76275          95.46550
3   2016           100.71300          96.18725
4   2017           100.00850          99.99750
5   2018           100.22675         102.18900
6   2019            97.90625         107.20625
7   2020            97.78950         114.53900
8   2021            99.48350         115.33175
9   2022            98.18900         120.61175
10  2023            98.03075         126.75750
11  2024            98.67175         130.99000


In [62]:
# draw core chart
year_slider = alt.binding_range(
    name='Show up to Year: ',
    min=int(df_year['Year'].min()),
    max=int(df_year['Year'].max()),
    step=1
)
year_sel = alt.selection_point(
    name='year_sel',               
    fields=['Year'],                
    bind=year_slider,        
    value={'Year': int(df_year['Year'].min())},  
    empty='none',                       
    on='input'                        
)

base = alt.Chart(df_year).encode(
    x=alt.X('Labor productivity:Q', title='Labor Productivity (Index, 2017=100)'),
    y=alt.Y('Unit labor costs:Q',  title='Unit Labor Costs (Index, 2017=100)', scale=alt.Scale(domain=[60, 140])),
    order=alt.Order('Year:O'),
    tooltip=[
      alt.Tooltip('Year:O', title='Year'),
      alt.Tooltip('Labor productivity:Q', format='.2f'),
      alt.Tooltip('Unit labor costs:Q', format='.2f'),
    ]
)

line   = base.mark_line(color='#1f77b4', strokeWidth=3).transform_filter(alt.datum.Year <= year_sel.Year)
points = base.mark_point(color='#ff7f0e', size=100, filled=True).transform_filter(alt.datum.Year <= year_sel.Year)


#year_labels = base.mark_text(dy=16, fontSize=14).encode(text='Year:O')

chart = (line + points).properties(
    width=800,
    title='Annual Connected Scatter: LP vs. ULC with Arrows'
)

chart

SchemaValidationError: '2013' is an invalid value for `Year`. Valid values are of type 'array'.

In [ ]:
import pandas as pd
import plotly.graph_objects as go



years = df_year['Year'].tolist()
min_x, max_x = df_year['Labor productivity'].min() - 1, df_year['Labor productivity'].max() + 1
min_y, max_y = df_year['Unit labor costs'].min() - 5, df_year['Unit labor costs'].max() + 5


init = df_year[df_year['Year'] == years[0]]
fig = go.Figure(
    data=[
        go.Scatter(
            x=init['Labor productivity'],
            y=init['Unit labor costs'],
            mode='lines+markers+text',
            text=init['Year'].astype(str),
            textposition='bottom center',
            marker=dict(size=12, color='orange'),
            line=dict(width=3, color='steelblue'),
        )
    ],
    layout=go.Layout(
        title="Annual Connected Scatter: LP vs. ULC",
        xaxis=dict(title="Labor Productivity (Index, 2017=100)", range=[min_x, max_x]),
        yaxis=dict(title="Unit Labor Costs (Index, 2017=100)", range=[min_y, max_y]),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 800, "redraw": True},
                                       "fromcurrent": True}])],
            pad={"r":10,"t":10}, showactive=False, x=0.1, y=0.05
        )],
        sliders=[dict(
            active=0,
            currentvalue={"prefix":"Year: "},
            pad={"t": 50},
            steps=[
                dict(method="animate",
                     args=[[str(year)],
                           {"frame": {"duration": 500, "redraw": True},
                            "mode": "immediate"}],
                     label=str(year))
                for year in years
            ]
        )]
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter(
                    x=df_year[df_year['Year'] <= year]['Labor productivity'],
                    y=df_year[df_year['Year'] <= year]['Unit labor costs'],
                    mode='lines+markers+text',
                    text=df_year[df_year['Year'] <= year]['Year'].astype(str),
                    textposition='bottom center',
                    marker=dict(size=12, color='orange'),
                    line=dict(width=3, color='steelblue'),
                )
            ],
            name=str(year)
        )
        for year in years
    ]
)

fig.show()
